In [8]:
def invert_mod_q(f):
    # f in ZZ[x], f^(-1) in ZZ[x]
    # ((f * f^(-1)) % (x^n + 1)) % q = 1
    
    T = Zx.change_ring(Integers(q)).quotient(x^n+1)
    return Zx(lift(1/T(f)))
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler
from math import sqrt
from random import randint
import time

def generate_trapdoor():
    # generate trapdoor matrix B, list[[]] representation
#     magic_constant = [k * (0.1 + 0.002 * i) for i in range(k)]

    B = [[None for j in range(k + 1)] for i in range(k + 1)]

    eta = 1
    for i in range(k):
        for j in range(k + 1):
            B[i][j] = Zx([randint(0,1) for i in range(n)])
#             sigma = magic_constant[i] * (q ** (1 / (k + 1)))
#             sig = sqrt(1 / (n * (k + 1 - i))) * sigma
#             B[i][j] = DiscreteGaussianDistributionPolynomialSampler(Zx, n, sig)()
    return B

def submatrix(M, row, col):
    nrow = len(M)
    ncol = len(M[0])
    rep = [[M[i][j] for j in range(ncol) if j != col] for i in range(nrow) if i != row]
    return rep

def matrix_invert_mod_q(f):
    # matrix: f in ZZ[x], f^(-1) in ZZ[x]
    # ((f * f^(-1)) % (x^n+1)) % q = 1
    
    f_det = f.determinant()
    f_det_inv = invert_mod_q(f_det)
    f_adj = f.adjugate()
    f_inv = f_adj * f_det_inv
    return f_det, f_adj, f_inv

def neg_mat(M):
    # M in [[]]
    # calculate the negative of a matrix
    return [[-M[i][j] for j in range(k)] for i in range(k)]

def xgcd_mod(f, g):
    # uf * f = df mod (x^n + 1)
    # ug * g = dg mod (x^n + 1)
    # df, dg in Z
    # uf, ug in Z[x]/(x^n + 1)
    df, uf, vf = (f % (x^n + 1)).xgcd(x^n + 1)
    dg, ug, vg = (g % (x^n + 1)).xgcd(x^n + 1)
    return df, dg, uf, ug

def to_bar(f):
    return Zx([f[0]]+[-f[n - i] for i in range(1, n)])

def cal_FG(f, g):
    rf, rg, _pf, _pg = xgcd_mod(f, g)
    d, u, v = rf.xgcd(rg)
    F = (q * v * (-1) * _pg) % (x^n + 1)
    G = (q * u * _pf) % (x^n + 1)
    return F, G

def convert_to_Zx(f):
    return Zx([int(i) for i in f])

def round_(f, num):
    # f in Zx
    # round(f/num) in Z[x]/(x^n + 1)
    return Zx([round(list(f)[i]/num) for i in range(n)])

def cal_ele_norm(ele):
    return math.sqrt(sum([float(i)**2 for i in ele]))

def cal_vec_norm(vec):
    k = len(vec)
    norm_arr = [cal_ele_norm(vec[i]) for i in range(k)]
    return max(norm_arr)

def cal_mat_norm(mat):
    k = len(mat)
    norm_arr = [cal_vec_norm(mat[i]) for i in range(k)]
    return max(norm_arr)

def mod_ntru_gen(fpr_ntru):
    while(true):
        B = generate_trapdoor()
        f = matrix(Zx, neg_mat(submatrix(B, k, 0)))
        g = matrix(Zx, k, 1, [B[j][0] for j in range(k)])
        try:
            f_det, f_adj, f_inv = matrix_invert_mod_q(f)  # f^(-1) mod q
        except (ZeroDivisionError, ValueError):
            print("inv mod reback", file = fpr_ntru)
            fpr_ntru.flush()
            continue
            
        h = ((f_inv * g) % (x^n + 1)) % q  # h = f^(-1) * g mod q

        # generate A = [1 h]
        A = [None for j in range(k + 1)]  
        for j in range(1, k + 1):
            A[j] = h[j - 1][0]
        A[0] = Zx([1])

        # generate F, G by f, g
        g_adjf = Zx(list((f_adj * g) % (x^n + 1))[0][0])
        rf, rg, _pf, _pg = xgcd_mod(f_det, g_adjf)
        
        if gcd(rf, rg)!=1 or gcd(rf, q)!=1:
            print("gcd reback", file = fpr_ntru)
            fpr_ntru.flush()
            continue
            
        F, G = cal_FG(f_det, g_adjf)

        # reduce F, G by F = F - k * f, G = G - k * g
        ffgg = (f_det * to_bar(f_det) + g_adjf * to_bar(g_adjf)) % (x^n + 1)
        fFgG = (F * to_bar(f_det) + G * to_bar(g_adjf)) % (x^n + 1)
        dfg, ufg, vfg = ffgg.xgcd(x^n + 1)
        k_fg = round_(convert_to_Zx((ufg * fFgG) % (x^n + 1)), int(dfg))
        F_ = F - (k_fg * (f_det % (x^n + 1))) % (x^n + 1)
        G_ = G - (k_fg * (g_adjf % (x^n + 1))) % (x^n + 1)

        # insert F, G into trapdoor matrix B
        B[k][0] = G_
        B[k][1] = -F_
        for i in range(2, k + 1):
            B[k][i] = Zx([0])
            
        # transform A, B from list to ring matrix in Z[x]/(x^n + 1)
        A_matZ = matrix(Zx, k + 1, 1, A)
        B_matZ = matrix(Zx, k + 1, k + 1, B)

        return A_matZ, B_matZ, B

def convert_matrix_to_list(mat):
    length = len(list(mat))
    width = len(list(list(mat)[0]))
    res = [[None for j in range(width)] for i in range(length)]
    for i in range(length):
        for j in range(width):
            res[i][j] = list(list(list(mat)[i])[j])
            while len(res[i][j]) < n:
                res[i][j].append(0)
    return res


from sage.modules.free_module_element import vector
def gs(M, fpr_ntru):
    if len(M) == 0 or len(M[0]) == 0:
        return M, matrix(ZZ, 0, 0, [])
    n = len(M)
    print(f"n = {n}", file = fpr_ntru)
    fpr_ntru.flush()
    Bstar = [M[0]]
    K = M[0].base_ring().fraction_field()
    zero = vector(K, len(M[0]))
    if Bstar[0] == zero:
        raise ValueError("linearly dependent input for module version of Gram-Schmidt")
    mu = matrix(K, n, n)
    for i in range(1, n):
        for j in range(i):
            print(f"i ={i}, j = {j}", file = fpr_ntru)
            fpr_ntru.flush()
            mu[i, j] = M[i].dot_product(Bstar[j]) / (Bstar[j].dot_product(Bstar[j]))
        Bstar.append(M[i] - sum(mu[i, j] * Bstar[j] for j in range(i)))
        if Bstar[i] == zero:
            raise ValueError("linearly dependent input for module version of Gram-Schmidt")
    return Bstar, mu

from datetime import datetime

n = 2**7
q = 8331299
k = 8

Zx.<x> = ZZ[]

with open(f'/home/cryptothesis/jwd/code/test/module_ntru_gen/param_1', 'w') as fpr_ntru:
    time1 = time.time()
    A, B, B_self = mod_ntru_gen(fpr_ntru)
    time2 = time.time()
    ntru_gen_time = time2 - time1
    print(f"ntrugen time = {ntru_gen_time}", file=fpr_ntru)
#     print("ntruGen end: " + str(datetime.now()), file = fpr_ntru)
 
    
a = [convert_matrix_to_list(A)[i][0] for i in range(len(list(A)))]
# Ta = convert_matrix_to_list(B)
import pickle
pickle.dump(a, open(f'/home/cryptothesis/jwd/code/data/param_1/a.pkl', 'wb'))
# pickle.dump(Ta, open(f'/home/cryptothesis/jwd/IBE_np/data/scheme1/data{n}/Ta.pkl', 'wb'))

# from sage.modules.misc import gram_schmidt
Qx.<x> = QQ[]
S = Qx.quotient(x^n + 1)
# B_matQ = matrix(S, k + 1, k + 1, B)

# try:
#     with open(f'/home/cryptothesis/jwd/modNTRU result/scheme2/n={n},q={q},k={k},G', 'w') as fpr_ntru:
#         print("Gram_Schmidt start: " + str(datetime.now()), file = fpr_ntru)
#         G, mu = gs(B_matQ.rows(), fpr_ntru)
#         print(f"GS norm = {cal_mat_norm(G)}", file=fpr_ntru)
#         fpr_ntru.flush()
#         print("Gram_Schmidt end: " + str(datetime.now()), file = fpr_ntru)
#         fpr_ntru.flush()
# except (ZeroDivisionError, ValueError):
#     print(1)


# Ta_gs = convert_matrix_to_list(G)
# with open(f'/home/cryptothesis/jwd/IBE/data{n}.py', 'w') as fpr:
#     print(f"a = {a}", file = fpr)
#     print(f"Ta = {Ta}", file = fpr)
#     print(f"Ta_gs = {Ta_gs}", file = fpr)
# pickle.dump(Ta_gs, open(f'/home/cryptothesis/jwd/IBE_np/data/scheme1/data{n}_/Ta_gs.pkl', 'wb'))

BS = B.change_ring(S)
BB_star = BS*(BS.T)
def ldl_star_decomp(M, fpr_ntru):
    m = k + 1
    L = zero_matrix(S, m, m)
    D = zero_matrix(S, m, m)
    D[0,0] = M[0,0]
    for i in range(1, m):
        for j in range(i):
            print(f"i ={i}, j = {j}", file = fpr_ntru)
            fpr_ntru.flush()
            L[i,j] = (M[i,j] - sum([L[i,k] * L[j,k] * D[k,k] for k in range(j)]))/D[j,j]
        D[i,i] = M[i,i] - sum([L[i,j] * L[i,j] * D[j,j] for j in range(i)])
    return L

with open(f'/home/cryptothesis/jwd/code/test/module_ntru_gen/param_1_gs', 'w') as fpr_ntru:
    time3 = time.time()
    mu = ldl_star_decomp(BB_star, fpr_ntru)
    R = Zx.quotient(x^n+1)
    B_R = B.change_ring(R)
#     print("start adj", file = fpr_ntru)
#     fpr_ntru.flush()
    adj_B = B_R.adjugate()
#     print("start det", file = fpr_ntru)
#     fpr_ntru.flush()
    det_B = B_R.determinant()
    w = S(det_B.lift())
    adj_BS = matrix(S, [[S(f.lift()) for f in row] for row in adj_B])
#     print("start inv", file = fpr_ntru)
#     fpr_ntru.flush()
    inv_B = matrix(S, [[f/w for f in row] for row in adj_BS])
#     print("end inv", file = fpr_ntru)
#     fpr_ntru.flush()
    time4 = time.time()
    gs_time = time4 - time3
    print(f"gs_time = {gs_time}", file = fpr_ntru)

pickle.dump(mu, open(f'/home/cryptothesis/jwd/code/data/param_1/mu.pkl', 'wb'))
pickle.dump(B, open(f'/home/cryptothesis/jwd/code/data/param_1/B.pkl', 'wb'))
pickle.dump(inv_B, open(f'/home/cryptothesis/jwd/code/data/param_1/inv_B.pkl', 'wb'))
